<a href="https://colab.research.google.com/github/dmarchignoli/haystack-rag/blob/main/simple_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dmarchignoli/haystack-rag.git

Cloning into 'haystack-rag'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 121 (delta 52), reused 83 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 71.59 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [6]:
%cd /content/haystack-rag/
!git pull

/content/haystack-rag
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 5.12 KiB | 1.71 MiB/s, done.
From https://github.com/dmarchignoli/haystack-rag
   ed1ad4a..e363e0c  main       -> origin/main
Updating ed1ad4a..e363e0c
Fast-forward
 resources/wp_paragraphs.txt | 39 +++++++++++++++++++++++++++++++++++++++
 resources/wp_queries.txt    | 20 ++++++++++++++++++++
 2 files changed, 59 insertions(+)
 create mode 100644 resources/wp_paragraphs.txt
 create mode 100644 resources/wp_queries.txt


In [3]:
!pip install haystack-ai sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.4/391.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 1.7 MB/s eta 0:00:00


In [4]:
from haystack import Document

texts = """
There are over 7,000 languages spoken around the world today.
Elephants have been observed to behave in a way that indicates a high level of self-awareness, such as recognizing themselves in mirrors.
In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the phenomenon of bioluminescent waves.
""".strip().split('\n')

documents = [Document(content=x) for x in texts]

for d in documents:
  print(d)


Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.')
Document(id=6f20658aeac3c102495b198401c1c0c2bd71d77b915820304d4fbc324b2f3cdb, content: 'Elephants have been observed to behave in a way that indicates a high level of self-awareness, such ...')
Document(id=7f225626ad1019b273326fbaf11308edfca6d663308a4a3533ec7787367d59a2, content: 'In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the ph...')


In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder

model_name="HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1"
text_embedder = SentenceTransformersTextEmbedder(model=model_name)
document_embedder = SentenceTransformersDocumentEmbedder(model=model_name)

text_embedder.warm_up()
document_embedder.warm_up()

print(text_embedder.run(texts[0])['embedding'])
print(document_embedder.run(documents[:1]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/600k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.0030116995330899954, -0.006784024182707071, -0.008190584369003773, -0.0049788388423621655, 0.004092300310730934, 7.893551082815975e-05, -0.008068575523793697, 0.0005798634374514222, -0.02081015333533287, 0.008193601854145527, -0.032294970005750656, 0.009576646611094475, -0.006836182903498411, -0.014892433770000935, -0.007112544495612383, 0.017983639612793922, -0.036985132843256, -0.021843085065484047, -0.03766949102282524, 0.0010961348889395595, -0.06414289027452469, -0.032925836741924286, -0.0120620122179389, 0.01574113219976425, 0.005689274985343218, 0.07852022349834442, 0.007477643899619579, 0.009653590619564056, 0.016210569068789482, -0.007366327103227377, -0.012833110056817532, -0.041939184069633484, 0.008757651783525944, -0.01100915390998125, 0.012921946123242378, 0.04111570492386818, 0.03296509385108948, 0.04146688058972359, 0.02009776048362255, -0.013988477177917957, -0.009523102082312107, 0.02056095562875271, 0.013438631780445576, 0.0009135070722550154, -0.02329723909497261

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', embedding: vector of size 896)]}


In [15]:
with open("resources/wp_paragraphs.txt", "r") as f:
    documents = [Document(content=line.strip()) for line in f if line.strip()]
with open("resources/wp_queries.txt", "r") as f:
    queries = [line.strip() for line in f if line.strip()]

dwe = document_embedder.run(documents)['documents']
qwe = [text_embedder.run(x) for x in queries]



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# prompt: produce a matrix with the cosine similarity of embedding values in dwe and qwe
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Extract embeddings from Document objects
document_embeddings = np.array([doc.embedding for doc in dwe])
query_embeddings = np.array(qwe)

# Calculate cosine similarity matrix
cosine_similarity(document_embeddings, query_embeddings)


In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryEmbeddingRetriever

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

documents_with_embeddings = document_embedder.run(documents)["documents"]
document_store.write_documents(documents_with_embeddings)
print(document_store.count_documents())
print(document_store.filter_documents({
    "field":"embedding", "operator":"==", "value":documents_with_embeddings[0].embedding}))

document_retriever = InMemoryEmbeddingRetriever(document_store=document_store)
found_docs = document_retriever.run(query_embedding=text_embedder.run(texts[0])['embedding'], top_k=2)
print(found_docs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

3
[Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', embedding: vector of size 896)]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', score: 0.9999999999994188), Document(id=7f225626ad1019b273326fbaf11308edfca6d663308a4a3533ec7787367d59a2, content: 'In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the ph...', score: 0.7183402171987093)]}


In [ ]:
from haystack import Pipeline

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", document_retriever)
query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

query = "How many languages are there?"

result = query_pipeline.run({"text_embedder": {"text": query}})

print(result['retriever']['documents'][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', score: 0.8211258197942695)


In [ ]:
!pip install qdrant-haystack

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.


In [ ]:
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever


document_store = QdrantDocumentStore(
    ":memory:",
    recreate_index=True,
    return_embedding=True,
    wait_result_from_api=True,
    embedding_dim=896
)
document_store.write_documents(documents_with_embeddings)
print(document_store.count_documents())
print(document_store.filter_documents())

retriever = QdrantEmbeddingRetriever(document_store=document_store)

found_docs = retriever.run(query_embedding=text_embedder.run(texts[0])['embedding'], top_k=1)
print(found_docs)

e = text_embedder.run("number of spoken languages")['embedding']
found_docs = retriever.run(query_embedding=e, top_k=1)
print(found_docs)


100it [00:00, 20422.16it/s]          

3
[Document(id=7f225626ad1019b273326fbaf11308edfca6d663308a4a3533ec7787367d59a2, content: 'In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the ph...', embedding: vector of size 896), Document(id=6f20658aeac3c102495b198401c1c0c2bd71d77b915820304d4fbc324b2f3cdb, content: 'Elephants have been observed to behave in a way that indicates a high level of self-awareness, such ...', embedding: vector of size 896), Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', embedding: vector of size 896)]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', score: 1.0000000682148074)]}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', score: 0.8507230803092024)]}
